# California Air Resources Board (CARB)

In [29]:
import requests
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup

### Parameters

In [5]:
URL = "https://ww2.arb.ca.gov/mrr-data"

headers = {
    "Authority": "ww2.arb.ca.gov",
    "Method": "GET",
    "Path": "/mrr-data",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "If-None-Match": "1689111602",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


### Tests

In [6]:
r = requests.get(URL, headers=headers)

In [17]:
soup = BeautifulSoup(r.content, "html.parser")
soup
# Get all hrefs and their text
href_dicts = []
hrefs = soup.find_all("a")
for href in hrefs:
    href_dict = {}
    text=  href.text
    url = href.get("href")

    if "xls" in url:     
        href_dict["name"] = text
        href_dict["url"] = f"https:{url}" 

        href_dicts.append(href_dict)
    

In [20]:
urls_df = pd.DataFrame.from_dict(href_dicts)

### Access one of the files

In [39]:
xlsx_url = urls_df['url'][1]

# Get data from first url
df = pd.read_excel(xlsx_url)

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [40]:
response = requests.get(xlsx_url)
xls_file = pd.ExcelFile(BytesIO(response.content))
# Create a dictionary of DataFrames, with sheet name as key
dataframes = {sheet_name: xls_file.parse(sheet_name) for sheet_name in xls_file.sheet_names}

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [42]:
dataframes

{'Introduction':     Unnamed: 0                 Unnamed: 1   
 0          NaN                        NaN  \
 1          NaN                        NaN   
 2          NaN                        NaN   
 3          NaN                        NaN   
 4          NaN  Released November 4, 2022   
 5          NaN                             
 6          NaN                        NaN   
 7          NaN                             
 8          NaN                        NaN   
 9          NaN                             
 10         NaN                        NaN   
 11         NaN                             
 12         NaN                        NaN   
 13         NaN                             
 14         NaN                        NaN   
 15         NaN                             
 16         NaN                        NaN   
 17         NaN                             
 18         NaN                        NaN   
 19         NaN                             
 20         NaN   

In [45]:
for key in dataframes.keys():
    if 'GHG Data' in key:
        key_of_interest = key

ghg_data = dataframes[key_of_interest]
# Remove rows with several NaNs
ghg_data = ghg_data.dropna(thresh=10)
# First row is the header
ghg_data.columns = ghg_data.iloc[0]
# Remove rows
longbeach_df = ghg_data.loc[ghg_data.loc[:, 'City'] == 'Long Beach'].reset_index(drop=True)
# Remove columns with NaNs
longbeach_df = longbeach_df.dropna(axis=1, how='all')
# Remove \n in column names
longbeach_df.columns = longbeach_df.columns.str.replace('\n', ' ')
longbeach_df

8,ARB ID,Facility Name,Report Year,"Total CO2e (combustion, process, vented, and supplier)",AEL,Emitter CO2e from Non-Biogenic Sources and CH4 and N2O from Biogenic Fuels,Emitter CO2 from Biogenic Fuels,Fuel Supplier CO2e from Non-Biogenic Fuels and CH4 and N2O from Biogenic Fuels,Fuel Supplier CO2 from Biogenic Fuels,Electricity Importer CO2e,...,Total Non-Covered Emissions,Emissions Data,Product Data,Verification Body,City,State,Zip Code,North American Industry Classification System (NAICS) Code and Description,U.S.EPA/ARB Subparts,Industry Sector
0,101321,"AES Alamitos, LLC",2020,921914.254313,No,921914.254313,0,0,0,0,...,0,Positive,NaN,Ashworth Leininger Group,Long Beach,CA,90803,221112 - Fossil Fuel Electric Power Generation,"C,D",In-State Electricity Generation
1,100002,Calciner,2020,277746.247885,No,277746.247885,0,0,0,0,...,1.279081,Positive,Positive,SCS Engineers,Long Beach,CA,90813,324199 - All Other Petroleum and Coal Products...,"C,Y",Other Combustion Source
2,104072,California Resources Production Corporation - ...,2020,251714.105823,No,251714.105823,0,0,0,0,...,4587.871197,Positive,Positive,"NSF Certification, LLC",Long Beach,CA,90802,211111 - Crude Petroleum and Natural Gas Extra...,"C,W",Oil and Gas Production
3,101697,"Gold Bond Building Products, LLC - Long Beach",2020,30381.132146,No,30381.132146,0,0,0,0,...,0,Positive,Positive,SCS Engineers,Long Beach,CA,90813,327420 - Gypsum Product Manufacturing,C,Other Combustion Source
4,5003,Long Beach Gas & Oil Dept (LBGO),2020,508754.121996,No,0,0,508754.121996,0,0,...,93355.526139,Positive,NaN,Locus Technologies,Long Beach,CA,90806,221210 - Natural Gas Distribution,NN,"Supplier of Natural Gas, NGL, or LPG"
5,100208,Long Beach Generating Station,2020,29890.804152,No,29890.804152,0,0,0,0,...,0,Positive,NaN,Ashworth Leininger Group,Long Beach,CA,90802,221112 - Fossil Fuel Electric Power Generation,D,In-State Electricity Generation
6,100187,Los Angeles Department of Water & Power-Haynes...,2020,1486395.78762,No,1486395.78762,0,0,0,0,...,1.782649,Positive,NaN,"Lincus, Inc.",Long Beach,CA,90803,221112 - Fossil Fuel Electric Power Generation,"C,D",In-State Electricity Generation
7,100063,Southeast Resource Recovery Facility (SERRF),2020,357951.444212,No,127893.344212,230058.1,0,0,0,...,230058.1,Positive,NaN,"Trinity Consultants, Inc.",Long Beach,CA,90802,562213 - Solid Waste Combustors and Incinerators,C,In-State Electricity Generation
